# Test S-CNN baseline

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import librosa
import glob
import sys

sys.path.insert(0,'../..')
from sed_endtoend.cnn.model import build_custom_cnn
from sed_endtoend.data_generator import DataGenerator

os.environ["CUDA_VISIBLE_DEVICES"]="1"

from params import *

# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

Using TensorFlow backend.


In [2]:
params = {'sequence_time': sequence_time, 'sequence_hop_time':sequence_hop_time,
          'label_list':label_list,'audio_hop':audio_hop, 'audio_win':audio_win,
          'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 
          'frames':frames,'get_annotations':get_annotations, 'dataset': dataset}

sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
labels = {}# Labels

test_files = sorted(glob.glob(os.path.join(audio_folder,'test', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    test_files = test_files[:load_subset]
    val_files = val_files[:load_subset]

test_labels = {}
test_mel = {}
val_labels = {}
val_mel = {}

for n,id in enumerate(test_files):
    labels[id] = os.path.join(label_folder, 'test',os.path.basename(id).replace('.wav','.txt'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

params['sequence_hop_time'] = sequence_time # To calculate F1_1s    
    
# Generators
print('Making test generator')
test_generator = DataGenerator(test_files, labels, **params)

print('Making validation generator')
validation_generator = DataGenerator(val_files, labels, **params)

print('Getting validation data')

_,_,mel_val,y_val = validation_generator.return_all()

print('Getting test data')

_,_,mel_test,y_test = test_generator.return_all()

Making test generator
Making validation generator
Getting validation data
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Getting test data
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [3]:
print('\nTesting model...')

sequence_samples = int(sequence_time*sr)

model = build_custom_cnn(n_freq_cnn=mel_bands, n_frames_cnn=sequence_frames,large_cnn=large_cnn)

weights_best_file = os.path.join(expfolder, 'weights_best.hdf5')
model.load_weights(weights_best_file)

model.summary()

y_test_predicted = model.predict(mel_test)
y_val_predicted = model.predict(mel_val)

np.save(os.path.join(expfolder, 'y_val_predicted.npy'),y_val_predicted)
np.save(os.path.join(expfolder, 'y_val.npy'),y_val)

np.save(os.path.join(expfolder, 'y_test_predict.npy'),y_test_predicted)
np.save(os.path.join(expfolder, 'y_test.npy'),y_test)

print("[Done]")

W1003 16:16:21.528877 140711985211136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1003 16:16:21.538780 140711985211136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1003 16:16:21.582764 140711985211136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1003 16:16:21.595330 140711985211136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecate


Testing model...


W1003 16:16:27.940439 140711985211136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W1003 16:16:28.087646 140711985211136 deprecation.py:506] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 44, 128)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 128)           512       
_________________________________________________________________
lambda_1 (Lambda)            (None, 44, 128, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 124, 128)      3328      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 62, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 62, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 58, 128)       409728    
__________